In [14]:
import numpy as np
import pandas as pd
import uproot
import csv
import os
import glob
#import doctest
#doctest.testmod(verbose=True)

#### Choose file to work with:

In [2]:
def get_file_name():
    
    file_name = input("\n Which .root file do you want to open?\n")
    if file_name is not '' and file_name.endswith('.root'):
        if os.path.isfile(file_name):
            return file_name
        else:
            print('\n No such file... Try again. \n')
            get_file_name()
    else:
        print ("\n No input or wrong file format given... Try again. \n")
        get_file_name()
        


#### Get the names of all the trees in the file

In [3]:
def get_tree_names(f_name):
    
    try:
        f = uproot.open(f_name)
        trees = f.keys()
        tree_names=[]

        print("\n Tree names successfully stored. They are: \n")
        for tree in trees:
            tree = str(tree)
            tree = tree[2:len(tree)-3]
            tree_names.append(tree)
            print(tree)
            print('\n')
            
        return tree_names

    except:
        print ("\n ERROR: Trees not found. Check your file before continuing...\n")



#### Unrolling the trees and writing them in separate files

In [45]:
def unroll_tree(file_name, ttree, of_name):
    files_out= [] 
    data = uproot.open(file_name)[ttree]
    names = data.keys()

    #data.arrays(names)
    out = pd.DataFrame.from_dict(data.arrays(names), dtype= str)
    out.to_csv(of_name)
    files_out.append(of_name)
    print ('\nCreated csv file ' + of_name)

In [46]:
def root_tree_to_csv():

    fname = get_file_name()
    woods = get_tree_names(fname)
    
    for tree in woods:    
        out_file_name = tree + '.csv'
        
        if os.path.isfile(out_file_name):
            answer = input('\nFile already exsists, do you want to overwrite?\n')
            no = ['n', 'No', 'no', 'NO']
            
            if answer in no:
                print( out_file_name + " will not be overwritten..")
            else:
                print ("Overwriting tree {} in file {}".format(tree, fname))
                unroll_tree(fname, tree, out_file_name)
                
        else: 
            print ("Writing tree {} in file {}".format(tree, fname))
            unroll_tree(fname, tree, out_file_name)

#### Adding 'label' column


In [66]:
def label_column_writer(infile, outfile):
    reader = csv.reader(open(infile, 'r'))
    writer = csv.writer(open(outfile, 'w'))
    headers = next(reader)
    headers.append("label")

    writer.writerow(headers)
    
    for row in reader:
        if infile == "signal_bbA_MA300tree.csv":
            row.append("signal")
            writer.writerow(row)
        else:
            row.append("background")
            writer.writerow(row)

In [67]:
def select_files():
    allfiles = [f for f in glob.glob('*.{}'.format('csv'))]
    files_to_modify = []

    #select files to modify
    for f in allfiles:
        answer = input('Do you want to add label column to the file "{}"? y or n'.format(f))
        if answer is 'y':
            files_to_modify.append(f)
        else:
            continue             
    return files_to_modify

In [68]:
def add_label_column():
    try:
        files = select_files()
        if len(files) is not 0:
            for file in files:
                out_file = "l_"+ file
                if os.path.isfile(out_file):
                    answer = input('\nA file named "{}" already exsists, do you want to overwrite it?\n'.format(out_file))
                    no = ['n', 'No', 'no', 'NO']
                    if answer in no:
                        print( out_file + " will not be overwritten..\n")
                    else: 
                        print( "\nOverwriting " + out_file) 
                        label_column_writer(file, out_file)
                else:
                    print( "\nWriting " + out_file)
                    label_column_writer(file, out_file)

    except:
        print('\n ERROR: No or invalid csv file found... \n')

#### merging files

In [15]:
def file_merger():
    
    try:

        all_filenames = [f for f in glob.glob('*.{}'.format('csv'))]
        #combine all files in the list
        combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ], sort = False)
        #pick a name for the new file
        file_out_name = input('\n How would you like to name the new file? \n')
        #export to csv
        combined_csv.to_csv( file_out_name, index=False, encoding='utf-8-sig')
    except:
        print('\n ERROR: No columns to parse from file...')

In [70]:
root_tree_to_csv()

KeyboardInterrupt: 

In [69]:
add_label_column()

Do you want to add label column to the file "signal_bbA_MA300tree.csv"? y or ny
Do you want to add label column to the file "analysis.csv"? y or nn
Do you want to add label column to the file "l_analysis.csv"? y or nn
Do you want to add label column to the file "l_signal_bbA_MA300tree.csv"? y or nn
Do you want to add label column to the file "bkg_DY_nlo1tree.csv"? y or ny
Do you want to add label column to the file "bkg_ttbar_nlotree.csv"? y or ny

A file named "l_signal_bbA_MA300tree.csv" already exsists, do you want to overwrite it?
y

Overwriting l_signal_bbA_MA300tree.csv

Writing l_bkg_DY_nlo1tree.csv

Writing l_bkg_ttbar_nlotree.csv


In [23]:
file_merger()


 How would you like to name the new file? 
analysis.csv
